In [8]:
import tensorflow as tf 
import pandas as pd
import numpy as np

In [9]:
# First get our data for csv.
df = pd.read_csv('StockData/BTCUSDT.csv')

In [10]:
# Gets all of our averages.

def get_averages(data):

    # only take close and open
    data = data[['Close', 'Open']]

    print(data)

    data['MA5'] = data['Close'].rolling(window=5).mean()

    data['MA20'] = data['Close'].rolling(window=20).mean()

    data['MA60'] = data['Close'].rolling(window=60).mean()

    # data['MA720'] = data['Close'].rolling(window=720).mean()

    # data['MA1440'] = data['Close'].rolling(window=1440).mean()

    data['MA10080'] = data['Close'].rolling(window=10080).mean()

    # data['MA131400'] = data['Close'].rolling(window=131400).mean()

    # Drop if even one of the values is na
    data = data.dropna()

    newdata = data[['Close', 'MA5', 'MA20', 'MA60', 'MA10080']].values

    return newdata


In [11]:
def normalise_averages(averages):
    
    # Normalise each column of the 2d array
    for i in range(averages.shape[1]):
        averages[:, i] = (averages[:, i] - averages[:, i].min()) / (averages[:, i].max() - averages[:, i].min())

    return averages

avg = get_averages(df)

print (avg.shape)

avg = normalise_averages(avg)

print(avg)

            Close      Open
0         4261.48   4261.48
1         4261.48   4261.48
2         4280.56   4280.56
3         4261.48   4261.48
4         4261.48   4261.48
...           ...       ...
3379522  39947.75  39945.00
3379523  39957.68  39947.74
3379524  39941.10  39957.69
3379525  39961.09  39941.11
3379526  39963.35  39961.09

[3379527 rows x 2 columns]


/tmp/ipykernel_10216/1393116232.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MA5'] = data['Close'].rolling(window=5).mean()
/tmp/ipykernel_10216/1393116232.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MA20'] = data['Close'].rolling(window=20).mean()
/tmp/ipykernel_10216/1393116232.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

(3369448, 5)
[[0.01959083 0.01946232 0.01921324 0.01862759 0.01269887]
 [0.01959083 0.01935492 0.01922354 0.01862294 0.01269863]
 [0.01959083 0.01935492 0.01921866 0.01861066 0.0126984 ]
 ...
 [0.56093105 0.56262899 0.56403235 0.56384613 0.60153578]
 [0.5612331  0.56267781 0.5640601  0.56384734 0.60153359]
 [0.56126724 0.56273348 0.56409752 0.56384538 0.6015315 ]]


In [17]:
def create_sequences(data, seq_length, future_steps):
    xs = []
    ys = []
    for i in range(len(data)-seq_length-future_steps):
        x = data[i:i+seq_length]
        # y is the mean over the next future_steps entries
        y = np.mean(data[i+seq_length:i+seq_length+future_steps, 0])
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 50  # Example: 50 days of past data
future_steps = 5  # Predicting the next 5 closing prices

x, y = create_sequences(avg, seq_length, future_steps)

print(x.shape, y.shape)

(3369393, 50, 5) (3369393,)


In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models,losses

train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Now we create a RNN model

# Create model

model = models.Sequential()
model.add(layers.LSTM(100, return_sequences=True, input_shape=(seq_length, x_train.shape[2])))
model.add(layers.LSTM(100, return_sequences=False))
model.add(layers.Dense(50))
model.add(layers.Dense(1))  # Output layer with future_steps units

model.compile(optimizer='adam',
              loss= losses.Huber(),
              metrics= [losses.MeanAbsoluteError()],
)

history = model.fit(x_train, y_train, epochs=3, 
                    validation_data=(x_test, y_test))


In [24]:
# More epochs
history = model.fit(x_train, y_train, epochs=10, 
                    validation_data=(x_test, y_test))

Epoch 1/10
84235/84235 [==============================] - 322s 4ms/step - loss: 6.1194e-07 - mean_absolute_error: 7.0115e-04 - val_loss: 1.0048e-06 - val_mean_absolute_error: 0.0014
Epoch 2/10
84235/84235 [==============================] - 327s 4ms/step - loss: 5.6240e-07 - mean_absolute_error: 6.7075e-04 - val_loss: 5.7895e-07 - val_mean_absolute_error: 9.9153e-04
Epoch 3/10
84235/84235 [==============================] - 323s 4ms/step - loss: 5.2451e-07 - mean_absolute_error: 6.4656e-04 - val_loss: 5.3648e-07 - val_mean_absolute_error: 9.6280e-04
Epoch 4/10
84235/84235 [==============================] - 327s 4ms/step - loss: 4.8190e-07 - mean_absolute_error: 6.1706e-04 - val_loss: 1.1095e-07 - val_mean_absolute_error: 3.0182e-04
Epoch 5/10
84235/84235 [==============================] - 324s 4ms/step - loss: 4.5676e-07 - mean_absolute_error: 5.9979e-04 - val_loss: 1.5956e-07 - val_mean_absolute_error: 3.8531e-04
Epoch 6/10
84235/84235 [==============================] - 331s 4ms/step - 

In [25]:
import matplotlib.pyplot as plt

# Extract loss values from the history object
loss = history.history['loss']
val_loss = history.history['val_loss']

# Create a range for the number of epochs
epochs = range(1, len(loss) + 1)

# Plot the loss values
plt.figure(figsize=(10, 6))
plt.plot(epochs, loss, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss, 'r*-', label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [26]:
# save the model
model.save('BTCUSDT.h5')

/nix/store/77j3ywj118fl439grgj9bcwrrvpjndpl-python3-3.11.8-env/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
# Every minute for the date of 01-03-2024 we use the model to predict the mean next 5 closing prices.

# First we need to get the data for that date
df = pd.read_csv('StockData/BTCUSDT.csv')

def get_averages(data):

    # only take close and open
    data = data[['Open time','Close', 'Open']]

    data['MA5'] = data['Close'].rolling(window=5).mean()

    data['MA20'] = data['Close'].rolling(window=20).mean()

    data['MA60'] = data['Close'].rolling(window=60).mean()

    data['MA10080'] = data['Close'].rolling(window=10080).mean()

    # Drop if even one of the values is na
    # data = data.dropna()

    newdata = data[['Open time', 'Close', 'MA5', 'MA20', 'MA60', 'MA10080']]

    return newdata


In [54]:

def normalise_averages(averages):
    
    # For every column in the pd dataframe we normalise
    for i in averages.columns[1:]:
        averages[i] = (averages[i] - averages[i].min()) / (averages[i].max() - averages[i].min())

    return averages

testing = get_averages(df)

testing = normalise_averages(testing)
# only take the data from '2024-01-03'
# get '2024-01-03 in unix time
# assigned regular string date


from datetime import datetime
import time

# # Normalise the values

# testing = testing.dropna()
# testing = 


testing['datetime'] = pd.to_datetime(testing['Open time'], unit='ms') 

# # take only the date of '2024-01-03'

testing = testing[testing['datetime'].dt.date == datetime.strptime('2024-01-03', '%Y-%m-%d').date()]

# # Now test these values with the model

# # Get the last seq_length values

last_values = testing[['Close', 'MA5', 'MA20', 'MA60', 'MA10080']].values[-seq_length:]

# # Run the values through the model

last_values = last_values.reshape(1, seq_length, 5)

# # Predict the mean of the next 5 closing prices

last_values = model.predict(last_values)

print(last_values)






/tmp/ipykernel_10216/2879855496.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MA5'] = data['Close'].rolling(window=5).mean()
/tmp/ipykernel_10216/2879855496.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MA20'] = data['Close'].rolling(window=20).mean()
/tmp/ipykernel_10216/2879855496.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

[[0.60125108 0.60341201 0.60602269 0.60662421 0.64103057]
 [0.60230875 0.60348963 0.6059657  0.60656169 0.64102938]
 [0.60158107 0.60341865 0.60587351 0.60648785 0.64102812]
 [0.60149706 0.60335637 0.60577255 0.60641041 0.64102681]
 [0.60158893 0.60338766 0.60567892 0.6063368  0.64102549]
 [0.60138087 0.60341371 0.60559304 0.60625909 0.64102416]
 [0.60173791 0.60329915 0.60552887 0.60619268 0.64102288]
 [0.60207848 0.60339897 0.60548185 0.60613898 0.64102162]
 [0.60237629 0.60357541 0.60544003 0.60609539 0.64102048]
 [0.60244776 0.60374776 0.60539373 0.60604604 0.64101931]
 [0.60266141 0.60400474 0.60534101 0.60600467 0.64101822]
 [0.60287929 0.60423379 0.6053501  0.60597101 0.64101714]
 [0.60257679 0.60433379 0.6053652  0.6059517  0.64101605]
 [0.60246634 0.60435187 0.60539016 0.60592657 0.64101493]
 [0.60256607 0.60437561 0.60540854 0.60590532 0.64101385]
 [0.60347657 0.6045392  0.60547854 0.60588816 0.64101289]
 [0.60359186 0.60468219 0.60556259 0.60587713 0.64101191]
 [0.60379342 0